In [1]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
model = load_model('gender_detection.model')

In [3]:
webcam = cv2.VideoCapture(0)
    
classes = ['man','woman']


while webcam.isOpened():


    status, frame = webcam.read()


    face, confidence = cv.detect_face(frame)


    for idx, f in enumerate(face):


        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]


        face_crop = np.copy(frame[startY:endY,startX:endX])

        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue


        face_crop = cv2.resize(face_crop, (96,96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)


        conf = model.predict(face_crop)[0]


        idx = np.argmax(conf)
        label = classes[idx]
        color = (255,0,0) if label == 'man' else (255, 153, 255)

        label = "{}: {:.2f}%".format(label, conf[idx] * 100)

        Y = startY - 10 if startY - 10 > 10 else startY + 10
        
        

        cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.75, color, 2)
        
        cv2.rectangle(frame, (startX,startY), (endX,endY), color, 2)
        

    cv2.imshow("gender detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()